In [0]:
from fastai import *
from fastai.tabular import *
import pandas as pd
from torchsummary import summary
import torch
from torch import nn
import imageio
import torch
import glob
from fastai.vision import *
import os
from torch import nn
import torch.nn.functional as F
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)
%cd "/content/drive/My Drive/automatic-asset-classification"
%ls "/content/drive/My Drive/automatic-asset-classification"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/automatic-asset-classification
data/


In [0]:
np.random.seed(3333)
torch.manual_seed(3333)
image_path = "/content/drive/My Drive/automatic-asset-classification/data/final_dataset"
size = 224
batchsize = 32

tfms = get_transforms(do_flip = True)
src = (ImageImageList.from_folder(image_path).split_by_rand_pct().label_from_func(lambda x: x))
data = (src.transform(tfms, size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(imagenet_stats, do_y = False))

In [0]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor = 2, mode = 'bilinear'),
            nn.Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(inplace=True)
        )
        
    def encode(self,x): return self.encoder(x)
    
    def decode(self,x): return torch.clamp(self.decoder(x), min = 0, max=1)
        

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return torch.clamp(decoded, min=0, max=1)

In [0]:
autoencoder = AutoEncoder()
learn = Learner(data, autoencoder, loss_func=F.mse_loss);
learn = learn.load('ae_12_31_3_2020')

In [0]:
train_batches = [item[0] for item in iter(data.train_dl)]
valid_batches = [item[0] for item in iter(data.valid_dl)]

In [0]:
autoencoder.eval()
encoded_train_batches = [];
encoded_valid_batches = [];

with torch.no_grad():

  for train in train_batches:
    encoded_train = autoencoder.encode(train);
    encoded_train_batches.append(encoded_train);
  
  for valid in valid_batches:
    encoded_valid = autoencoder.encode(valid);
    encoded_valid_batches.append(encoded_valid);

In [0]:
del train_batches
del valid_batches
del data
del src
del autoencoder
del learn

# Autoencoder - Layer 2

In [0]:
class AutoEncoder2(nn.Module):
    def __init__(self):
        super(AutoEncoder2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(64, 112, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.BatchNorm2d(112, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor = 2, mode = 'bilinear'),
            nn.Conv2d(112, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.Tanh()
        )
    def encode(self,x): return self.encoder(x)
    def decode(self,x): return self.decoder(x)
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

Create the training data which is the encoded output from the first autoencoder.

In [10]:
encoded_train_batches = torch.cat([batch.cpu() for batch in encoded_train_batches])
encoded_valid_batches = torch.cat([batch.cpu() for batch in encoded_valid_batches])
print(encoded_train_batches.shape), print(encoded_valid_batches.shape)

torch.Size([384, 64, 112, 112])
torch.Size([96, 64, 112, 112])


(None, None)

Define the parameters.

In [0]:
bs = 32
epoch = 100

Training loop:

In [0]:
autoencoder2 = AutoEncoder2()

In [0]:
class encoderDataset():
  def __init__(self, x): self.x,self.y = x,x
  def __len__(self): return len(self.x)
  def __getitem__(self, i): return self.x[i],self.y[i]

In [0]:
train_ds,valid_ds = encoderDataset(encoded_train_batches),encoderDataset(encoded_valid_batches)

In [0]:
from torch.utils.data import DataLoader

In [0]:
train_dl = DataLoader(train_ds, bs, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, bs, shuffle=False)

In [0]:
lr = 0.01
opt = torch.optim.SGD(autoencoder2.parameters(), lr=lr)

for epoch in range(epochs):

  for xb,yb in train_dl:
    autoencoder2.train()

    loss = F.mse_loss(pred, yb)

    loss.backward()
    opt.step()
    opt.zero_grad()


    model.eval()
    with torch.no_grad():
        tot_loss,tot_acc = 0.,0.
        for xb,yb in valid_dl:
            pred = model(xb)
            tot_loss += loss_func(pred,yb)
            tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)


In [0]:
#encoded_train_batches = torch.cat([batch.cpu() for batch in encoded_train_batches])
#encoded_valid_batches = torch.cat([batch.cpu() for batch in encoded_valid_batches])

encoded_train_batches = np.concatenate([batch.cpu().numpy() for batch in encoded_train_batches])
encoded_valid_batches = np.concatenate([batch.cpu().numpy() for batch in encoded_valid_batches])